In [1]:
import polars as pl
import numpy as np
import pandas as pd
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    for col in df.columns:
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))
    return df
dataPath = "/kaggle/input/home-credit-credit-risk-model-stability/"
base_train = pl.read_csv(dataPath + "csv_files/test/test_base.csv")
base_train.head()

case_id,date_decision,MONTH,WEEK_NUM
i64,str,i64,i64
57543,"""2021-05-14""",202201,100
57549,"""2022-01-17""",202201,100
57551,"""2020-11-27""",202201,100
57552,"""2020-11-27""",202201,100
57569,"""2021-12-20""",202201,100


In [2]:
#Depth=0
static_train=pl.read_csv(dataPath + "csv_files/test/test_static_cb_0.csv")
static_train=set_table_dtypes(static_train)
drop_list=["assignmentdate_238D","assignmentdate_4527235D","assignmentdate_4955616D","dateofbirth_342D","for3years_128L","for3years_504L","for3years_584L","formonth_118L","formonth_206L","formonth_535L","forquarter_1017L","forquarter_462L","forquarter_634L","fortoday_1092L","forweek_1077L","forweek_528L","forweek_601L","foryear_618L","foryear_818L","foryear_850L","pmtaverage_3A","pmtaverage_4527227A_","pmtaverage_4955615A","pmtcount_4527229L","pmtcount_4955617L","pmtcount_693L","riskassesment_302T","riskassesment_940T"]
static_train=static_train.drop(drop_list)

for col in static_train.columns:
    if static_train[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]:
        static_train = static_train.with_columns(pl.col(col).fill_null(0))
    elif static_train[col].dtype in [pl.Utf8, pl.Categorical]:
        static_train = static_train.with_columns(pl.col(col).fill_null("Unknown"))
static_train.head()

case_id,birthdate_574D,contractssum_5085716L,dateofbirth_337D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_4527227A,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,secondquarter_766L,thirdquarter_1082L
i64,str,f64,str,f64,f64,f64,f64,f64,str,str,str,f64,f64,str,str,f64,f64,str,f64,str,str,str,str,f64,f64
57543,"""Unknown""",151364.0,"""1980-11-01""",2.0,4.0,1.0,8.0,2.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",4.0,9.0,"""38c061ee""","""a55475b1""",8.0,0.0,"""Unknown""",0.0,"""Unknown""","""Unknown""","""Unknown""","""2021-05-28""",2.0,3.0
57549,"""Unknown""",1.5631e6,"""1959-11-01""",6.0,9.0,3.0,12.0,4.0,"""2fc785b2""","""39a0853f""","""a55475b1""",9.0,5.0,"""a7fcb6e5""","""a55475b1""",12.0,0.0,"""Unknown""",0.0,"""Unknown""","""Unknown""","""Unknown""","""2022-01-31""",8.0,2.0
57551,"""Unknown""",2926195.3,"""1982-05-01""",1.0,3.0,1.0,4.0,1.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",3.0,2.0,"""3439d993""","""a55475b1""",4.0,0.0,"""Unknown""",0.0,"""Unknown""","""Unknown""","""Unknown""","""2020-12-11""",5.0,5.0
57552,"""Unknown""",747031.73,"""1955-11-01""",2.0,2.0,0.0,5.0,0.0,"""2fc785b2""","""a55475b1""","""a55475b1""",3.0,2.0,"""a55475b1""","""a55475b1""",5.0,0.0,"""Unknown""",0.0,"""Unknown""","""Unknown""","""Unknown""","""2020-12-11""",7.0,1.0
57569,"""Unknown""",0.0,"""1949-09-01""",4.0,4.0,1.0,4.0,4.0,"""2fc785b2""","""717ddd49""","""a55475b1""",0.0,0.0,"""3439d993""","""a55475b1""",4.0,0.0,"""Unknown""",0.0,"""Unknown""","""Unknown""","""Unknown""","""2022-01-03""",1.0,3.0


In [3]:
#Depth=2
applprev_train=pl.read_csv(dataPath+"csv_files/test/test_applprev_2.csv")
applprev_train=applprev_train.drop("credacc_cards_status_52L")
for col in applprev_train.columns:
    if applprev_train[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]:
        applprev_train = applprev_train.with_columns(pl.col(col).fill_null(0))
    elif applprev_train[col].dtype in [pl.Utf8, pl.Categorical]:
        applprev_train = applprev_train.with_columns(pl.col(col).fill_null("Unknown"))
applprev_train=applprev_train.drop(["num_group1","num_group2"])
applprev_train.head()

case_id,cacccardblochreas_147M,conts_type_509L
i64,str,str
57543,"""a55475b1""","""PRIMARY_MOBILE…"
57543,"""a55475b1""","""Unknown"""
57543,"""a55475b1""","""PRIMARY_MOBILE…"
57543,"""a55475b1""","""Unknown"""
57543,"""a55475b1""","""PRIMARY_MOBILE…"


In [4]:
#Depth=1
other_train=pl.read_csv(dataPath+"csv_files/test/test_other_1.csv")
other_train=set_table_dtypes(other_train)
other_train=other_train.group_by("case_id").agg(pl.col("amtdebitincoming_4809443A").max().alias("amtdebitincoming_max"),
                                               pl.col("amtdebitoutgoing_4809440A").max().alias("amtdebitoutgoing_max"),
                                               pl.col("amtdepositbalance_4809441A").max().alias("amtdepositbalance_max"),
                                               pl.col("amtdepositincoming_4809444A").max().alias("amtdepositincoming_max"),
                                               pl.col("amtdepositoutgoing_4809442A").max().alias("amtdepositoutgoing_max"))
other_train.head()

case_id,amtdebitincoming_max,amtdebitoutgoing_max,amtdepositbalance_max,amtdepositincoming_max,amtdepositoutgoing_max
i64,f64,f64,f64,f64,f64
57657,17171.4,19970.0,228.40001,0.0,1.8000001
57644,0.0,60.4,0.0,0.0,0.0
57543,0.0,0.0,1579.6,6900.0,33.4
57667,9930.8,9930.8,0.0,0.0,0.0
57549,34933.402,34820.0,0.0,0.0,0.0


In [5]:
#Depth=1
tax_train=pl.read_csv(dataPath+"csv_files/test/test_tax_registry_a_1.csv")
tax_train=set_table_dtypes(tax_train)
tax_num=tax_train.group_by("case_id").agg(pl.col("amount_4527230A").max().alias("amount_max"))
tax_cat=tax_train.select(["case_id","name_4527232M","recorddate_4527225D","num_group1"]).drop("num_group1")
tax_train=tax_num.join(tax_cat,how="inner",on="case_id")
tax_train.head()

case_id,amount_max,name_4527232M,recorddate_4527225D
i64,f64,str,str
57675,5061.8003,"""d7cce9e8""","""2022-01-07"""
57675,5061.8003,"""ff9eb829""","""2022-01-07"""
57675,5061.8003,"""ff9eb829""","""2022-01-07"""
57675,5061.8003,"""d7cce9e8""","""2022-01-07"""
57675,5061.8003,"""ff9eb829""","""2022-01-07"""


In [6]:
#Depth=1
credit_train=pl.read_csv(dataPath+"csv_files/test/test_credit_bureau_b_1.csv")
credit_train=set_table_dtypes(credit_train)
credit_train=credit_train.drop("periodicityofpmts_997L")
for col in credit_train.columns:
    if credit_train[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]:
        credit_train = credit_train.with_columns(pl.col(col).fill_null(0))
    elif credit_train[col].dtype in [pl.Utf8, pl.Categorical]:
        credit_train = credit_train.with_columns(pl.col(col).fill_null("Unknown"))


numerical_columns = [col for col in credit_train.drop(["case_id","num_group1"]).columns if credit_train[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]]
credit_num=credit_train.groupby("case_id").agg([pl.col(col).max().alias(f"{col}_max") for col in numerical_columns])
cat_columns=[col for col in credit_train.columns if credit_train[col].dtype in [pl.Utf8, pl.Categorical]]
credit_cat=credit_train.select(["case_id","num_group1"]+cat_columns).drop("num_group1")
credit_train=credit_num.join(credit_cat,how="inner",on="case_id")
credit_train.head()

/tmp/ipykernel_244/592483517.py:13: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  credit_num=credit_train.groupby("case_id").agg([pl.col(col).max().alias(f"{col}_max") for col in numerical_columns])


case_id,amount_1115A_max,credlmt_1052A_max,credlmt_228A_max,credlmt_3940954A_max,credquantity_1099L_max,credquantity_984L_max,debtpastduevalue_732A_max,debtvalue_227A_max,dpd_550P_max,dpd_733P_max,dpdmax_851P_max,dpdmaxdatemonth_804T_max,dpdmaxdateyear_742T_max,installmentamount_644A_max,installmentamount_833A_max,instlamount_892A_max,interesteffectiverate_369L_max,interestrateyearly_538L_max,maxdebtpduevalodued_3940955A_max,numberofinstls_810L_max,overdueamountmax_950A_max,overdueamountmaxdatemonth_494T_max,overdueamountmaxdateyear_432T_max,pmtdaysoverdue_1135P_max,pmtnumpending_403L_max,residualamount_1093A_max,residualamount_127A_max,residualamount_3940956A_max,totalamount_503A_max,totalamount_881A_max,classificationofcontr_1114M,contractdate_551D,contractmaturitydate_151D,contractst_516M,contracttype_653M,credor_3940957M,lastupdate_260D,periodicityofpmts_997M,pmtmethod_731M,purposeofcred_722M,subjectrole_326M,subjectrole_43M
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str
57675,1.488e6,220598.0,10000.0,200598.0,2.0,19.0,0.0,1.433179e6,0.0,0.0,37403.0,12.0,2019.0,0.0,1.433179e6,17030.264,0.0,0.0,0.0,181.0,10.6,12.0,2019.0,0.0,160.0,0.0,182816.69,165095.89,1.488e6,480262.6,"""01f63ac8""","""2020-04-22""","""2035-04-23""","""04bf6e27""","""60e784d6""","""74bd67a8""","""2022-01-02""","""a0b598e4""","""e914c86c""","""60c73645""","""a55475b1""","""a55475b1"""
57675,1.488e6,220598.0,10000.0,200598.0,2.0,19.0,0.0,1.433179e6,0.0,0.0,37403.0,12.0,2019.0,0.0,1.433179e6,17030.264,0.0,0.0,0.0,181.0,10.6,12.0,2019.0,0.0,160.0,0.0,182816.69,165095.89,1.488e6,480262.6,"""ea6782cc""","""2020-06-01""","""2022-06-02""","""7241344e""","""1c9c5356""","""b619fa46""","""2021-12-27""","""a55475b1""","""a55475b1""","""60c73645""","""ab3c25cf""","""ab3c25cf"""
57675,1.488e6,220598.0,10000.0,200598.0,2.0,19.0,0.0,1.433179e6,0.0,0.0,37403.0,12.0,2019.0,0.0,1.433179e6,17030.264,0.0,0.0,0.0,181.0,10.6,12.0,2019.0,0.0,160.0,0.0,182816.69,165095.89,1.488e6,480262.6,"""ea6782cc""","""2012-09-29""","""2022-09-29""","""7241344e""","""1c9c5356""","""b619fa46""","""2021-12-24""","""a55475b1""","""a55475b1""","""60c73645""","""ab3c25cf""","""ab3c25cf"""
57754,1932619.4,132032.0,38000.0,76800.0,4.0,11.0,0.0,1473622.4,0.0,0.0,0.0,12.0,2020.0,0.0,1473622.4,8052.5796,0.0,39.0,0.0,240.0,0.0,12.0,2020.0,0.0,183.0,0.0,68059.91,50039.508,1932619.4,575772.44,"""01f63ac8""","""2017-06-09""","""2037-06-09""","""7241344e""","""c6678a8f""","""3169c2b9""","""2022-03-14""","""a0b598e4""","""f6e26148""","""9e302002""","""a55475b1""","""a55475b1"""
57754,1932619.4,132032.0,38000.0,76800.0,4.0,11.0,0.0,1473622.4,0.0,0.0,0.0,12.0,2020.0,0.0,1473622.4,8052.5796,0.0,39.0,0.0,240.0,0.0,12.0,2020.0,0.0,183.0,0.0,68059.91,50039.508,1932619.4,575772.44,"""ea6782cc""","""2018-11-18""","""2022-11-18""","""7241344e""","""1c9c5356""","""b619fa46""","""2022-06-03""","""a55475b1""","""a55475b1""","""60c73645""","""ab3c25cf""","""ab3c25cf"""


In [7]:
#Depth=1
deposit_train=pl.read_csv(dataPath+"csv_files/test/test_deposit_1.csv")
deposit_train=set_table_dtypes(deposit_train)
deposit_train = deposit_train.with_columns(pl.col("contractenddate_991D").fill_null("Unknown"))
deposit_num=deposit_train.group_by("case_id").agg(pl.col("amount_416A").max().alias("deposit_amount_max"))
deposit_cat=deposit_train.select(["case_id","contractenddate_991D","num_group1","openingdate_313D"]).drop("num_group1")
deposit_train=deposit_num.join(deposit_cat,how="inner",on="case_id")
deposit_train.head()

case_id,deposit_amount_max,contractenddate_991D,openingdate_313D
i64,f64,str,str
57644,0.0,"""2016-11-29""","""2013-11-29"""
57644,0.0,"""Unknown""","""2012-08-01"""
57644,0.0,"""Unknown""","""2013-08-08"""
57701,0.0,"""2017-09-05""","""2014-09-06"""
57708,262.558,"""2019-04-21""","""2016-04-21"""


In [8]:
#Depth=1
person_train=pl.read_csv(dataPath+"csv_files/test/test_person_1.csv")
person_train=set_table_dtypes(person_train)
drop_list=["birthdate_87D","childnum_185L","gender_992L","housetype_905L","housingtype_772L","isreference_387L","maritalst_703L","role_993L"]
person_train=person_train.drop(drop_list)
for col in person_train.columns:
    if person_train[col].dtype == pl.Boolean:
        person_train = person_train.with_columns(person_train[col].cast(pl.Int32).alias(col))
for col in person_train.columns:
    if person_train[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]:
        person_train = person_train.with_columns(pl.col(col).fill_null(0))
    elif person_train[col].dtype in [pl.Utf8, pl.Categorical]:
        person_train = person_train.with_columns(pl.col(col).fill_null("Unknown"))
        
        
numerical_columns = [col for col in person_train.drop(["case_id","num_group1"]).columns if person_train[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]]
person_num=person_train.groupby("case_id").agg([pl.col(col).max().alias(f"{col}_max") for col in numerical_columns])
cat_columns=[col for col in person_train.columns if person_train[col].dtype in [pl.Utf8, pl.Categorical]]
person_cat=person_train.select(["case_id","num_group1"]+cat_columns).drop("num_group1")
person_train=person_num.join(person_cat,how="inner",on="case_id")
person_train.head()

/tmp/ipykernel_244/2441444581.py:17: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  person_num=person_train.groupby("case_id").agg([pl.col(col).max().alias(f"{col}_max") for col in numerical_columns])


case_id,contaddr_matchlist_1032L_max,contaddr_smempladdr_334L_max,mainoccupationinc_384A_max,personindex_1023L_max,persontype_1072L_max,persontype_792L_max,remitter_829L_max,safeguarantyflag_411L_max,birth_259D,contaddr_district_15M,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,empl_employedtotal_800L,empl_industry_691L,empladdr_district_926M,empladdr_zipcode_114M,familystate_447L,incometype_1044T,language1_981M,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,role_1084L,sex_738L,type_25L
i64,i32,i32,f64,f64,f64,f64,i32,i32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
57543,0,0,34000.0,1.0,5.0,5.0,0,0,"""1980-11-01""","""P107_155_111""","""P91_47_168""","""P33_146_175""","""Unknown""","""Unknown""","""Unknown""","""a55475b1""","""a55475b1""","""SINGLE""","""SALARIED_GOVT""","""a55475b1""","""P107_155_111""","""P91_47_168""","""Unknown""","""Unknown""","""CL""","""F""","""PRIMARY_MOBILE…"
57543,0,0,34000.0,1.0,5.0,5.0,0,0,"""Unknown""","""a55475b1""","""a55475b1""","""a55475b1""","""Unknown""","""Unknown""","""Unknown""","""a55475b1""","""a55475b1""","""Unknown""","""Unknown""","""a55475b1""","""a55475b1""","""a55475b1""","""SIBLING""","""SIBLING""","""PE""","""Unknown""","""PHONE"""
57549,0,0,49800.0,1.0,5.0,5.0,0,1,"""1959-11-01""","""a55475b1""","""a55475b1""","""P106_81_188""","""Unknown""","""Unknown""","""Unknown""","""a55475b1""","""a55475b1""","""SINGLE""","""RETIRED_PENSIO…","""a55475b1""","""P121_131_159""","""P111_112_180""","""Unknown""","""Unknown""","""CL""","""F""","""PRIMARY_MOBILE…"
57549,0,0,49800.0,1.0,5.0,5.0,0,1,"""Unknown""","""a55475b1""","""a55475b1""","""a55475b1""","""Unknown""","""Unknown""","""Unknown""","""a55475b1""","""a55475b1""","""Unknown""","""Unknown""","""a55475b1""","""a55475b1""","""a55475b1""","""COLLEAGUE""","""COLLEAGUE""","""PE""","""Unknown""","""PHONE"""
57551,0,0,59600.0,0.0,1.0,1.0,0,0,"""1982-05-01""","""P11_36_178""","""P97_107_128""","""a55475b1""","""2002-01-08""","""Unknown""","""Unknown""","""a55475b1""","""a55475b1""","""Unknown""","""SALARIED_GOVT""","""P10_39_147""","""P11_36_178""","""P97_107_128""","""Unknown""","""Unknown""","""CL""","""F""","""PRIMARY_MOBILE…"


In [9]:
#Depth=1
debitcard_train=pl.read_csv(dataPath+"csv_files/test/test_debitcard_1.csv")
debitcard_train=set_table_dtypes(debitcard_train)
debitcard_train=debitcard_train.drop(["last180dayaveragebalance_704A","last180dayturnover_1134A","last30dayturnover_651A"])
debitcard_train = debitcard_train.with_columns(pl.col("openingdate_857D").fill_null("Unknown"))
debitcard_train=debitcard_train.drop("num_group1")
debitcard_train.head()

case_id,openingdate_857D
i64,str
57644,"""2013-11-29"""
57644,"""2012-08-01"""
57644,"""2013-08-08"""
57701,"""2014-09-06"""
57708,"""2016-04-21"""


In [10]:
df=base_train.join(static_train,how="left",on="case_id"
                  ).join(applprev_train,how="left",on="case_id"
                ).join(other_train,how="left",on="case_id"
                ).join(tax_train,how="left",on="case_id"
                ).join(credit_train,how="left",on="case_id"
                ).join(deposit_train,how="left",on="case_id"
                ).join(person_train,how="left",on="case_id"
                ).join(debitcard_train,how="left",on="case_id")
df.head()

import gc
del static_train, applprev_train, other_train, tax_train, credit_train, deposit_train, person_train, debitcard_train
gc.collect()

0

In [11]:
cols_to_select = ['amtdebitincoming_max', 'amtdebitoutgoing_max', 'amtdepositbalance_max', 'amtdepositincoming_max', 'amtdepositoutgoing_max', 'amount_1115A_max', 'credlmt_1052A_max', 'credlmt_228A_max', 'credlmt_3940954A_max', 'credquantity_1099L_max', 'credquantity_984L_max', 'debtpastduevalue_732A_max', 'debtvalue_227A_max', 'dpd_550P_max', 'dpd_733P_max', 'dpdmax_851P_max', 'dpdmaxdatemonth_804T_max', 'dpdmaxdateyear_742T_max', 'installmentamount_644A_max', 'installmentamount_833A_max', 'instlamount_892A_max', 'interesteffectiverate_369L_max', 'interestrateyearly_538L_max', 'maxdebtpduevalodued_3940955A_max', 'numberofinstls_810L_max', 'overdueamountmax_950A_max', 'overdueamountmaxdatemonth_494T_max', 'overdueamountmaxdateyear_432T_max', 'pmtdaysoverdue_1135P_max', 'pmtnumpending_403L_max', 'residualamount_1093A_max', 'residualamount_127A_max', 'residualamount_3940956A_max', 'totalamount_503A_max', 'totalamount_881A_max', 'classificationofcontr_1114M', 'contractdate_551D', 'contractmaturitydate_151D', 'contractst_516M', 'contracttype_653M', 'credor_3940957M', 'lastupdate_260D', 'periodicityofpmts_997M', 'pmtmethod_731M', 'purposeofcred_722M', 'subjectrole_326M', 'subjectrole_43M', 'deposit_amount_max', 'contractenddate_991D', 'openingdate_313D', 'openingdate_857D']
df = df.drop(cols_to_select)
for col in df.columns:
    if df[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]:
        df = df.with_columns(pl.col(col).fill_null(0))
    elif df[col].dtype in [pl.Utf8, pl.Categorical]:
        df = df.with_columns(pl.col(col).fill_null("Unknown"))
df=df.drop(["name_4527232M","contaddr_district_15M","contaddr_zipcode_807M","empladdr_district_926M","empladdr_zipcode_114M","registaddr_district_1083M","registaddr_zipcode_184M",
           "relationshiptoclient_415T","relationshiptoclient_642T"])
df.head()

case_id,date_decision,MONTH,WEEK_NUM,birthdate_574D,contractssum_5085716L,dateofbirth_337D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_4527227A,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,secondquarter_766L,thirdquarter_1082L,cacccardblochreas_147M,conts_type_509L,amount_max,recorddate_4527225D,contaddr_matchlist_1032L_max,contaddr_smempladdr_334L_max,mainoccupationinc_384A_max,personindex_1023L_max,persontype_1072L_max,persontype_792L_max,remitter_829L_max,safeguarantyflag_411L_max,birth_259D,education_927M,empl_employedfrom_271D,empl_employedtotal_800L,empl_industry_691L,familystate_447L,incometype_1044T,language1_981M,role_1084L,sex_738L,type_25L
i64,str,i64,i64,str,f64,str,f64,f64,f64,f64,f64,str,str,str,f64,f64,str,str,f64,f64,str,f64,str,str,str,str,f64,f64,str,str,f64,str,i32,i32,f64,f64,f64,f64,i32,i32,str,str,str,str,str,str,str,str,str,str,str
57543,"""2021-05-14""",202201,100,"""Unknown""",151364.0,"""1980-11-01""",2.0,4.0,1.0,8.0,2.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",4.0,9.0,"""38c061ee""","""a55475b1""",8.0,0.0,"""Unknown""",0.0,"""Unknown""","""Unknown""","""Unknown""","""2021-05-28""",2.0,3.0,"""a55475b1""","""PRIMARY_MOBILE…",0.0,"""Unknown""",0,0,34000.0,1.0,5.0,5.0,0,0,"""1980-11-01""","""P33_146_175""","""Unknown""","""Unknown""","""Unknown""","""SINGLE""","""SALARIED_GOVT""","""a55475b1""","""CL""","""F""","""PRIMARY_MOBILE…"
57543,"""2021-05-14""",202201,100,"""Unknown""",151364.0,"""1980-11-01""",2.0,4.0,1.0,8.0,2.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",4.0,9.0,"""38c061ee""","""a55475b1""",8.0,0.0,"""Unknown""",0.0,"""Unknown""","""Unknown""","""Unknown""","""2021-05-28""",2.0,3.0,"""a55475b1""","""PRIMARY_MOBILE…",0.0,"""Unknown""",0,0,34000.0,1.0,5.0,5.0,0,0,"""Unknown""","""a55475b1""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""a55475b1""","""PE""","""Unknown""","""PHONE"""
57543,"""2021-05-14""",202201,100,"""Unknown""",151364.0,"""1980-11-01""",2.0,4.0,1.0,8.0,2.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",4.0,9.0,"""38c061ee""","""a55475b1""",8.0,0.0,"""Unknown""",0.0,"""Unknown""","""Unknown""","""Unknown""","""2021-05-28""",2.0,3.0,"""a55475b1""","""Unknown""",0.0,"""Unknown""",0,0,34000.0,1.0,5.0,5.0,0,0,"""1980-11-01""","""P33_146_175""","""Unknown""","""Unknown""","""Unknown""","""SINGLE""","""SALARIED_GOVT""","""a55475b1""","""CL""","""F""","""PRIMARY_MOBILE…"
57543,"""2021-05-14""",202201,100,"""Unknown""",151364.0,"""1980-11-01""",2.0,4.0,1.0,8.0,2.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",4.0,9.0,"""38c061ee""","""a55475b1""",8.0,0.0,"""Unknown""",0.0,"""Unknown""","""Unknown""","""Unknown""","""2021-05-28""",2.0,3.0,"""a55475b1""","""Unknown""",0.0,"""Unknown""",0,0,34000.0,1.0,5.0,5.0,0,0,"""Unknown""","""a55475b1""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""Unknown""","""a55475b1""","""PE""","""Unknown""","""PHONE"""
57543,"""2021-05-14""",202201,100,"""Unknown""",151364.0,"""1980-11-01""",2.0,4.0,1.0,8.0,2.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",4.0,9.0,"""38c061ee""","""a55475b1""",8.0,0.0,"""Unknown""",0.0,"""Unknown""","""Unknown""","""Unknown""","""2021-05-28""",2.0,3.0,"""a55475b1""","""PRIMARY_MOBILE…",0.0,"""Unknown""",0,0,34000.0,1.0,5.0,5.0,0,0,"""1980-11-01""","""P33_146_175""","""Unknown""","""Unknown""","""Unknown""","""SINGLE""","""SALARIED_GOVT""","""a55475b1""","""CL""","""F""","""PRIMARY_MOBILE…"


In [12]:
date_list=['date_decision','birthdate_574D','dateofbirth_337D','responsedate_1012D','responsedate_4527233D','responsedate_4917613D','recorddate_4527225D','birth_259D','empl_employedfrom_271D']
for column_name in date_list:
    # Replace "Unknown" with None and immediately convert to date
    df = df.with_columns(
        pl.when(pl.col(column_name).cast(pl.String) == "Unknown")
        .then(None)
        .otherwise(pl.col(column_name))
        .alias(column_name))
    df = df.with_columns(
    pl.col(column_name).cast(pl.String).str.strptime(pl.Date, "%Y-%m-%d"))
    
date_list=['date_decision','birthdate_574D','dateofbirth_337D','responsedate_1012D','responsedate_4527233D','responsedate_4917613D','recorddate_4527225D','birth_259D','empl_employedfrom_271D']
for i in date_list:
    df = df.with_columns(
    (df.select(pl.col(i).rank(method='dense')).get_column(i) - 1).alias(i))

    
df = df.with_columns(
        pl.when(pl.col("contractssum_5085716L").cast(pl.String) == "Unknown")
        .then(None)
        .otherwise(pl.col(column_name))
        .alias("contractssum_5085716L"))
df=df.with_columns(pl.col("contractssum_5085716L").cast(pl.Float64,strict=False).alias("contractssum_5085716L"))


for col in df.columns:
    if df[col].dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64]:
        df = df.with_columns(pl.col(col).fill_null(0))
df.head()

case_id,date_decision,MONTH,WEEK_NUM,birthdate_574D,contractssum_5085716L,dateofbirth_337D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_4527227A,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,secondquarter_766L,thirdquarter_1082L,cacccardblochreas_147M,conts_type_509L,amount_max,recorddate_4527225D,contaddr_matchlist_1032L_max,contaddr_smempladdr_334L_max,mainoccupationinc_384A_max,personindex_1023L_max,persontype_1072L_max,persontype_792L_max,remitter_829L_max,safeguarantyflag_411L_max,birth_259D,education_927M,empl_employedfrom_271D,empl_employedtotal_800L,empl_industry_691L,familystate_447L,incometype_1044T,language1_981M,role_1084L,sex_738L,type_25L
i64,u32,i64,i64,u32,f64,u32,f64,f64,f64,f64,f64,str,str,str,f64,f64,str,str,f64,f64,str,f64,str,u32,u32,u32,f64,f64,str,str,f64,u32,i32,i32,f64,f64,f64,f64,i32,i32,u32,str,u32,str,str,str,str,str,str,str,str
57543,3,202201,100,0,0.0,6,2.0,4.0,1.0,8.0,2.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",4.0,9.0,"""38c061ee""","""a55475b1""",8.0,0.0,"""Unknown""",0.0,"""Unknown""",0,0,3,2.0,3.0,"""a55475b1""","""PRIMARY_MOBILE…",0.0,0,0,0,34000.0,1.0,5.0,5.0,0,0,4,"""P33_146_175""",0,"""Unknown""","""Unknown""","""SINGLE""","""SALARIED_GOVT""","""a55475b1""","""CL""","""F""","""PRIMARY_MOBILE…"
57543,3,202201,100,0,0.0,6,2.0,4.0,1.0,8.0,2.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",4.0,9.0,"""38c061ee""","""a55475b1""",8.0,0.0,"""Unknown""",0.0,"""Unknown""",0,0,3,2.0,3.0,"""a55475b1""","""PRIMARY_MOBILE…",0.0,0,0,0,34000.0,1.0,5.0,5.0,0,0,0,"""a55475b1""",0,"""Unknown""","""Unknown""","""Unknown""","""Unknown""","""a55475b1""","""PE""","""Unknown""","""PHONE"""
57543,3,202201,100,0,0.0,6,2.0,4.0,1.0,8.0,2.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",4.0,9.0,"""38c061ee""","""a55475b1""",8.0,0.0,"""Unknown""",0.0,"""Unknown""",0,0,3,2.0,3.0,"""a55475b1""","""Unknown""",0.0,0,0,0,34000.0,1.0,5.0,5.0,0,0,4,"""P33_146_175""",0,"""Unknown""","""Unknown""","""SINGLE""","""SALARIED_GOVT""","""a55475b1""","""CL""","""F""","""PRIMARY_MOBILE…"
57543,3,202201,100,0,0.0,6,2.0,4.0,1.0,8.0,2.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",4.0,9.0,"""38c061ee""","""a55475b1""",8.0,0.0,"""Unknown""",0.0,"""Unknown""",0,0,3,2.0,3.0,"""a55475b1""","""Unknown""",0.0,0,0,0,34000.0,1.0,5.0,5.0,0,0,0,"""a55475b1""",0,"""Unknown""","""Unknown""","""Unknown""","""Unknown""","""a55475b1""","""PE""","""Unknown""","""PHONE"""
57543,3,202201,100,0,0.0,6,2.0,4.0,1.0,8.0,2.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",4.0,9.0,"""38c061ee""","""a55475b1""",8.0,0.0,"""Unknown""",0.0,"""Unknown""",0,0,3,2.0,3.0,"""a55475b1""","""PRIMARY_MOBILE…",0.0,0,0,0,34000.0,1.0,5.0,5.0,0,0,4,"""P33_146_175""",0,"""Unknown""","""Unknown""","""SINGLE""","""SALARIED_GOVT""","""a55475b1""","""CL""","""F""","""PRIMARY_MOBILE…"


In [13]:
string_columns=['description_5085714M', 'education_1103M', 'education_88M', 'maritalst_385M', 'maritalst_893M', 'requesttype_4525192L', 'cacccardblochreas_147M', 'conts_type_509L', 'education_927M', 'empl_employedtotal_800L', 'empl_industry_691L', 'familystate_447L', 'incometype_1044T', 'language1_981M', 'role_1084L', 'sex_738L', 'type_25L']
for col in string_columns:
    categories_dict = {'description_5085714M': ['a55475b1', '2fc785b2', 'Unknown'], 'education_1103M': ['c8e1a1d0', 'a55475b1', 'Unknown', '717ddd49', '39a0853f', '6b2ae0fa'], 'education_88M': ['6b2ae0fa', 'a34a13c8', 'c8e1a1d0', 'a55475b1', 'Unknown', '717ddd49'], 'maritalst_385M': ['3439d993', '38c061ee', 'ecd83604', 'a7fcb6e5', 'b6cabe76', 'a55475b1', 'Unknown'], 'maritalst_893M': ['Unknown', '977b2a70', 'ecd83604', '1a19667c', 'e18430ff', 'a55475b1', '46b968c3'], 'requesttype_4525192L': ['PENSION_6', 'SOCIAL_6', 'Unknown', 'DEDUCTION_6'], 'cacccardblochreas_147M': ['Unknown', 'P23_105_103', 'P17_56_144', 'a55475b1', 'P201_63_60', 'P41_107_150', 'P19_60_110', 'P133_119_56', 'P33_145_161', 'P127_74_114'], 'conts_type_509L': ['SECONDARY_MOBILE', 'Unknown', 'ALTERNATIVE_PHONE', 'PRIMARY_MOBILE', 'PHONE', 'WHATSAPP', 'HOME_PHONE', 'PRIMARY_EMAIL', 'EMPLOYMENT_PHONE'], 'education_927M': ['P17_36_170', 'P157_18_172', 'P106_81_188', 'a55475b1', 'P97_36_170', 'P33_146_175'], 'empl_employedtotal_800L': ['MORE_ONE', 'Unknown', 'LESS_ONE', 'MORE_FIVE'], 'empl_industry_691L': ['ARMY_POLICE', 'OTHER', 'POST_TELCO', 'EDUCATION', 'RECRUITMENT', 'TRADE', 'WELNESS', 'GOVERNMENT', 'IT', 'HEALTH', 'GAMING', 'Unknown', 'REAL_ESTATE', 'ART_MEDIA', 'MANUFACTURING', 'FINANCE', 'AGRICULTURE', 'LAWYER', 'CATERING', 'TOURISM', 'INSURANCE', 'TRANSPORTATION', 'MARKETING', 'MINING', 'CHARITY_RELIGIOUS'], 'familystate_447L': ['Unknown', 'LIVING_WITH_PARTNER', 'SINGLE', 'MARRIED', 'WIDOWED', 'DIVORCED'], 'incometype_1044T': ['HANDICAPPED', 'EMPLOYED', 'RETIRED_PENSIONER', 'OTHER', 'PRIVATE_SECTOR_EMPLOYEE', 'SELFEMPLOYED', 'SALARIED_GOVT', 'HANDICAPPED_2', 'HANDICAPPED_3'], 'language1_981M': ['P10_39_147', 'a55475b1', 'P209_127_106'], 'role_1084L': ['EM', 'CL'], 'sex_738L': ['M', 'F'], 'type_25L': ['WHATSAPP', 'SECONDARY_MOBILE', 'HOME_PHONE', 'PRIMARY_MOBILE', 'PHONE', 'PRIMARY_EMAIL']}
    categories = categories_dict[col]
    for category in categories:
        df = df.with_columns((pl.col(col) == category).cast(pl.UInt8).alias(f"{col}_{category}"))

df=df.drop(string_columns)
xxx = df[['WEEK_NUM', 'case_id']]
df.head()

case_id,date_decision,MONTH,WEEK_NUM,birthdate_574D,contractssum_5085716L,dateofbirth_337D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,firstquarter_103L,fourthquarter_440L,numberofqueries_373L,pmtaverage_4527227A,pmtscount_423L,pmtssum_45A,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,secondquarter_766L,thirdquarter_1082L,amount_max,recorddate_4527225D,contaddr_matchlist_1032L_max,contaddr_smempladdr_334L_max,mainoccupationinc_384A_max,personindex_1023L_max,persontype_1072L_max,persontype_792L_max,remitter_829L_max,safeguarantyflag_411L_max,birth_259D,empl_employedfrom_271D,description_5085714M_a55475b1,description_5085714M_2fc785b2,…,empl_industry_691L_AGRICULTURE,empl_industry_691L_LAWYER,empl_industry_691L_CATERING,empl_industry_691L_TOURISM,empl_industry_691L_INSURANCE,empl_industry_691L_TRANSPORTATION,empl_industry_691L_MARKETING,empl_industry_691L_MINING,empl_industry_691L_CHARITY_RELIGIOUS,familystate_447L_Unknown,familystate_447L_LIVING_WITH_PARTNER,familystate_447L_SINGLE,familystate_447L_MARRIED,familystate_447L_WIDOWED,familystate_447L_DIVORCED,incometype_1044T_HANDICAPPED,incometype_1044T_EMPLOYED,incometype_1044T_RETIRED_PENSIONER,incometype_1044T_OTHER,incometype_1044T_PRIVATE_SECTOR_EMPLOYEE,incometype_1044T_SELFEMPLOYED,incometype_1044T_SALARIED_GOVT,incometype_1044T_HANDICAPPED_2,incometype_1044T_HANDICAPPED_3,language1_981M_P10_39_147,language1_981M_a55475b1,language1_981M_P209_127_106,role_1084L_EM,role_1084L_CL,sex_738L_M,sex_738L_F,type_25L_WHATSAPP,type_25L_SECONDARY_MOBILE,type_25L_HOME_PHONE,type_25L_PRIMARY_MOBILE,type_25L_PHONE,type_25L_PRIMARY_EMAIL
i64,u32,i64,i64,u32,f64,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,u32,u32,u32,f64,f64,f64,u32,i32,i32,f64,f64,f64,f64,i32,i32,u32,u32,u8,u8,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
57543,3,202201,100,0,0.0,6,2.0,4.0,1.0,8.0,2.0,4.0,9.0,8.0,0.0,"""Unknown""",0.0,0,0,3,2.0,3.0,0.0,0,0,0,34000.0,1.0,5.0,5.0,0,0,4,0,0,1,…,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0
57543,3,202201,100,0,0.0,6,2.0,4.0,1.0,8.0,2.0,4.0,9.0,8.0,0.0,"""Unknown""",0.0,0,0,3,2.0,3.0,0.0,0,0,0,34000.0,1.0,5.0,5.0,0,0,0,0,0,1,…,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
57543,3,202201,100,0,0.0,6,2.0,4.0,1.0,8.0,2.0,4.0,9.0,8.0,0.0,"""Unknown""",0.0,0,0,3,2.0,3.0,0.0,0,0,0,34000.0,1.0,5.0,5.0,0,0,4,0,0,1,…,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0
57543,3,202201,100,0,0.0,6,2.0,4.0,1.0,8.0,2.0,4.0,9.0,8.0,0.0,"""Unknown""",0.0,0,0,3,2.0,3.0,0.0,0,0,0,34000.0,1.0,5.0,5.0,0,0,0,0,0,1,…,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
57543,3,202201,100,0,0.0,6,2.0,4.0,1.0,8.0,2.0,4.0,9.0,8.0,0.0,"""Unknown""",0.0,0,0,3,2.0,3.0,0.0,0,0,0,34000.0,1.0,5.0,5.0,0,0,4,0,0,1,…,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0


In [14]:
selected_feature_names = ['date_decision', 'contractssum_5085716L', 'dateofbirth_337D',
       'days120_123L', 'days180_256L', 'days30_165L', 'days360_512L',
       'days90_310L', 'firstquarter_103L', 'fourthquarter_440L',
       'numberofqueries_373L', 'pmtaverage_4527227A', 'pmtscount_423L',
       'pmtssum_45A', 'responsedate_1012D', 'responsedate_4527233D',
       'responsedate_4917613D', 'secondquarter_766L', 'thirdquarter_1082L',
       'amount_max', 'recorddate_4527225D', 'mainoccupationinc_384A_max',
       'persontype_792L_max', 'birth_259D', 'empl_employedfrom_271D',
       'description_5085714M_2fc785b2', 'education_1103M_39a0853f',
       'education_1103M_6b2ae0fa', 'requesttype_4525192L_Unknown',
       'requesttype_4525192L_DEDUCTION_6', 'cacccardblochreas_147M_Unknown',
       'conts_type_509L_Unknown', 'education_927M_P97_36_170',
       'education_927M_P33_146_175', 'empl_employedtotal_800L_MORE_ONE',
       'empl_industry_691L_OTHER', 'incometype_1044T_RETIRED_PENSIONER',
       'incometype_1044T_PRIVATE_SECTOR_EMPLOYEE', 'sex_738L_M', 'sex_738L_F']

df = df[selected_feature_names]

df = df.with_columns(
        pl.when(pl.col("pmtscount_423L") == "Unknown")
        .then(None)
        .otherwise(pl.col(column_name))
        .alias("pmtscount_423L"))




In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score 
import joblib
import numpy as np
scaler = joblib.load('/kaggle/input/homecredit-david/scaler_model.pkl')
model = joblib.load('/kaggle/input/homecredit-david/decision_tree_model.pkl')
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

df = df[['date_decision', 'dateofbirth_337D', 'days120_123L', 'days180_256L',
       'days30_165L', 'days360_512L', 'days90_310L', 'firstquarter_103L',
       'fourthquarter_440L', 'numberofqueries_373L', 'pmtaverage_4527227A',
       'pmtscount_423L', 'pmtssum_45A', 'responsedate_1012D',
       'responsedate_4527233D', 'responsedate_4917613D', 'secondquarter_766L',
       'thirdquarter_1082L', 'amount_max', 'recorddate_4527225D',
       'mainoccupationinc_384A_max', 'persontype_792L_max', 'birth_259D',
       'empl_employedfrom_271D', 'education_1103M_6b2ae0fa',
       'education_1103M_39a0853f', 'education_927M_P97_36_170',
       'empl_employedtotal_800L_MORE_ONE', 'empl_industry_691L_OTHER',
       'incometype_1044T_PRIVATE_SECTOR_EMPLOYEE']]
df = df.fillna(0)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/conda/lib/python3.10/site-packages/sklearn/util

In [16]:
df['case_id'] = xxx['case_id']
df = df.drop_duplicates(subset=['case_id'])
df = df.set_index("case_id")

y_pred = pd.Series(model.predict_proba(df)[:, 1], index=df.index)

df_subm = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/sample_submission.csv")
df_subm = df_subm.set_index("case_id")

df_subm["score"] = y_pred
df_subm.to_csv("submission.csv")